This notebook deploys DaoSwap on Optimism testnet, adds orders and executes `getTotals`. 

All transactions can be found on https://goerli-optimism.etherscan.io/ by the transaction hash, or by account address 0x6531e2613bbbEEcd898356F7b9bEfBaEfd42804B.

Cost analysis according to the transactions made in this notebook can be found here https://docs.google.com/document/d/1Kxu6gfUTtfLKaw2O_SBn_iBfpORhlRFMB4uJ4RpA4Hs/edit#

# Imports

In [1]:
import os
import sys
import time
import pprint

from eth_account import Account
from eth_account.signers.local import LocalAccount
import web3
from web3 import Web3, EthereumTesterProvider
from web3.middleware import construct_sign_and_send_raw_middleware
import solcx

# Set-up Alchemy & Accounts

In [40]:
alchemy_key = os.environ["ALCHEMY_KEY"]
alchemy_url = f"https://opt-goerli.g.alchemy.com/v2/{alchemy_key}"
w3 = Web3(Web3.HTTPProvider(alchemy_url))

print(w3.isConnected())

True


In [41]:
account_addr = "0x6531e2613bbbEEcd898356F7b9bEfBaEfd42804B"
private_key = os.environ["PRIVATE_KEY"]

account: LocalAccount = Account.from_key(private_key)
w3.middleware_onion.add(construct_sign_and_send_raw_middleware(account))

print(f"Your hot wallet address is {account.address}")

Your hot wallet address is 0x6531e2613bbbEEcd898356F7b9bEfBaEfd42804B


# Functions

In [2]:
def compile_source_file(file_path):
    with open(file_path, 'r') as f:
        source = f.read()
    return solcx.compile_source(source, output_values=["abi", "bin", "bin-runtime"])

# Deployment

In [26]:
compiled = compile_source_file("contracts/DaoSwap.sol")
contract_interface = compiled["<stdin>:DaoSwap"]

In [12]:
contractName = "DaoSwap";
tokens = ["0xfb6115445Bff7b52FeB98650C87f44907E58f802"];
swapPeriodInSecs = 300;
swapRandomizationInSecs = 5;
feesAsPct = 5;
priceMode = 3;
priceOracle = "0x547a514d5e3769680Ce22B2361c10Ea13619e8a9";
swapMode = 1;

func=w3.eth.contract(
    abi=contract_interface["abi"],
    bytecode=contract_interface["bin"]).constructor(
        contractName, 
        tokens,
        swapPeriodInSecs,
        swapRandomizationInSecs,
        feesAsPct,
        priceMode,
        priceOracle,
        swapMode)

In [ ]:
tx_hash = func.transact({"from": account_addr})

In [17]:
address = w3.eth.get_transaction_receipt(tx_hash)["contractAddress"]

In [27]:
# DaoSwap contract address is 0xa07c906a6c9eDCb21a03943586e8920D2d716E3D
contract = w3.eth.contract(address=address, abi=contract_interface["abi"])

In [49]:
token_to_buy = "0xfb6115445Bff7b52FeB98650C87f44907E58f802"
amount_of_token_18dec = 1000000000000000000
limit_price = 54430000000000000
price_of_token = 54330000000000000
deposit_addr = "0x5B38Da6a701c568545dCfcB03FcB875f56beddC4"

# Create buy order.
# This is not explicitly depicted in the notebook, but 11 orders were created.
buy_order_tx_hash = contract.functions.buyOrder(token_to_buy,
                           amount_of_token_18dec,
                           limit_price,
                           deposit_addr).transact(
                    {"from": account_addr, "value": 54430000000000000})

In [52]:
# Execute `getTotals`.
# This thansaction for this function also was also sent several times, 
# so we can compare the gas spent for iterating 9, 10 and 11 orders.
get_totals_tx_hash = contract.functions.getTotals(price_of_token).transact( {"from": account_addr})

In [53]:
get_totals_tx_hash

HexBytes('0x9737643eba2b9161724ac70b0ed66a2fb357c03c63d3d20aa8ea6186de5586da')

In [51]:
# Get the current length of orders array.
len(contract.functions.getOrders().call({"from": account_addr}))

11